# About Crim Intervals 

- See more at https://github.com/HCDigitalScholarship/intervals/blob/master/README.md

## How to Select Pieces

- For local file(s), use: `corpus = CorpusBase(['/Users/rfreedma/MEI/CRIM_Intervals_Tests/Brumel_Complete.mei'])`
- For remote file, use: `corpus = CorpusBase(['https://crimproject.org/mei/CRIM_Model_0008.mei', 'https://crimproject.org/mei/CRIM_Mass_0005_5.mei'])`


## Notes about the Various Parameters

- **Length of the Soggetto**: into_patterns([vectors.semitone_intervals], 5) The **number** in this command represents the **minimum number of vectors to find**. 5 vectors is 6 notes.

### Chromatic vs Diatonic 
- **Chromatic** uses `into_patterns([vectors.semitone_intervals], 5)`
- **Diatonic** uses `into_patterns([vectors.generic_intervals], 5)`

### Exact vs Close  
- **Exact** is exact in *all* ways find_exact_matches(patterns, 2). The **number** in this command represents the **minimum number of matching melodies needed before reporting**. This allows us to filter for common or uncommon soggetti.
- **Close** matches allow for melodic variation (see more below). `find_close_matches(patterns, 2, 1)`
. The **first number** in this command is the minimum number of melodies needed before reporting; the **second number** is threshold needed in order to find a match. Lower number = very similar; higher number = less similar

### More about Close Matches  
- The **threshold for close matches** is determined by the third number called in the method. We select two patterns, then compare *each vector in each pattern successively*. The "differences" between each vector are summed. If that value is below the threshold specified, we consider the two patterns closely matched.
- The format of the method call is  `find_close_matches(the array you get from into_patterns, minimum matches needed to be displayed, threshold for close match)`.

### About Rhythmic Durations

- For `find_close_matches` and `find_exact_matches`, rhythmic variation/duration is displayed, but **not** factored into the calculation of matching.
- **Incremental Offset** calculates the intervals using a fixed offset between notes, no matter their actual duration.  Use this to ignore passing tones or other ornaments.  The offsets are expressed in multiples of the quarter note (Offset = 1 samples at quarter note; Offset = 2 at half note, etc). Set with `vectors = IntervalBase(corpus.note_list_incremental_offset(2))`

### Dataframe Preview vs Full Results in Browser

- Comment out the first of these for preview only.  Include for full results

    - `#pd.set_option("display.max_rows", None, "display.max_columns", None)`
    - `return pd.DataFrame(match_data)`
    
### To Set CSV Output

- Specify file name here `pd.Series(match_data).to_csv("Match_data_10_13.csv")`

***

## Search with Options

- Piece or Corpus
- Actual or Incremental Durations
- Chromatic or Diatonic
- Exact or Close
- Classify

***


### Load Crim Intervals and Pandas

In [93]:
from crim_intervals import *
import pandas as pd

***

### Selected Piece or Corpus 

A Note About Local Files
Use /content/your_file_name.mei for any local file you've uploaded

Otherwise provide URL

To load CRIM files directly from Git:

In [ ]:
# complete corpus

all_crim = ['https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Mass_0001_1.mei', 
'https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Mass_0001_2.mei', 
'https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Mass_0001_3.mei', 
'https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Mass_0001_4.mei', 
'https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Mass_0001_5.mei', 
'https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Mass_0002_1.mei', 
'https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Mass_0002_2.mei', 
'https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Mass_0002_3.mei', 
'https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Mass_0002_4.mei', 
'https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Mass_0002_5.mei', 
'https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Mass_0003_1.mei', 
'https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Mass_0003_2.mei', 
'https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Mass_0003_3.mei', 
'https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Mass_0003_4.mei', 
'https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Mass_0003_5.mei', 
'https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Mass_0004_1.mei', 
'https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Mass_0004_2.mei', 
'https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Mass_0004_3.mei', 
'https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Mass_0004_4.mei', 
'https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Mass_0004_5.mei', 
'https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Mass_0005_1.mei', 
'https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Mass_0005_2.mei', 
'https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Mass_0005_3.mei', 
'https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Mass_0005_4.mei', 
'https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Mass_0005_5.mei', 
'https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Mass_0006_1.mei', 
'https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Mass_0006_2.mei', 
'https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Mass_0006_3.mei', 
'https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Mass_0006_4.mei', 
'https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Mass_0006_5.mei', 
'https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Mass_0007_1.mei', 
'https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Mass_0007_2.mei', 
'https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Mass_0007_3.mei', 
'https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Mass_0007_4.mei', 
'https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Mass_0007_5.mei', 
'https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Mass_0008_1.mei', 
'https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Mass_0008_2.mei', 
'https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Mass_0008_3.mei', 
'https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Mass_0008_4.mei', 
'https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Mass_0008_5.mei', 
'https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Mass_0009_1.mei', 
'https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Mass_0009_2.mei', 
'https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Mass_0009_3.mei', 
'https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Mass_0009_4.mei', 
'https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Mass_0009_5.mei', 
'https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Mass_0010_1.mei', 
'https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Mass_0010_2.mei', 
'https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Mass_0010_3.mei', 
'https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Mass_0010_4.mei', 
'https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Mass_0010_5.mei', 
'https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Mass_0011_1.mei', 
'https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Mass_0011_2.mei', 
'https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Mass_0011_3.mei', 
'https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Mass_0011_4.mei', 
'https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Mass_0011_5.mei', 
'https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Mass_0012_1.mei', 
'https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Mass_0012_2.mei', 
'https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Mass_0012_3.mei', 
'https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Mass_0012_4.mei', 
'https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Mass_0012_5.mei', 
'https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Mass_0013_1.mei', 
'https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Mass_0013_2.mei', 
'https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Mass_0013_3.mei', 
'https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Mass_0013_4.mei', 
'https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Mass_0013_5.mei', 
'https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Mass_0014_1.mei', 
'https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Mass_0014_2.mei', 
'https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Mass_0014_3.mei', 
'https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Mass_0014_4.mei', 
'https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Mass_0014_5.mei', 
'https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Mass_0015_1.mei', 
'https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Mass_0015_2.mei', 
'https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Mass_0015_3.mei', 
'https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Mass_0015_4.mei', 
'https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Mass_0015_5.mei', 
'https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Mass_0016_1.mei', 
'https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Mass_0016_2.mei', 
'https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Mass_0016_3.mei', 
'https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Mass_0016_4.mei', 
'https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Mass_0016_5.mei', 
'https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Mass_0017_1.mei', 
'https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Mass_0017_2.mei', 
'https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Mass_0017_3.mei', 
'https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Mass_0017_4.mei', 
'https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Mass_0017_5.mei', 
'https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Mass_0018_1.mei', 
'https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Mass_0018_2.mei', 
'https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Mass_0018_3.mei', 
'https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Mass_0018_4.mei', 
'https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Mass_0018_5.mei', 
'https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Mass_0019_1.mei', 
'https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Mass_0019_2.mei', 
'https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Mass_0019_3.mei', 
'https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Mass_0019_4.mei', 
'https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Mass_0019_5.mei', 
'https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Mass_0020_1.mei', 
'https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Mass_0020_2.mei', 
'https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Mass_0020_3.mei', 
'https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Mass_0020_4.mei', 
'https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Mass_0020_5.mei', 
'https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Mass_0021_1.mei', 
'https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Mass_0021_2.mei', 
'https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Mass_0021_3.mei', 
'https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Mass_0021_4.mei', 
'https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Mass_0021_5.mei', 
'https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Mass_0022_2.mei', 
'https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Model_0001.mei', 
'https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Model_0008.mei', 
'https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Model_0009.mei', 
'https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Model_0010.mei', 
'https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Model_0011.mei', 
'https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Model_0012.mei', 
'https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Model_0013.mei', 
'https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Model_0014.mei', 
'https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Model_0015.mei', 
'https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Model_0016.mei', 
'https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Model_0017.mei', 
'https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Model_0019.mei', 
'https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Model_0020.mei', 
'https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Model_0021.mei', 
'https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Model_0023.mei', 
'https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Model_0024.mei', 
'https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Model_0025.mei', 
'https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Model_0026.mei']

In [94]:
#corpus = CorpusBase(['/Users/rfreedma/MEI/A_MEI_Tests/Sandrin_Doulce_RF_12_15_20.mei'])
#url = 'https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/'
#corpus_files = all_crim
corpus_files = ['https://crimproject.org/mei/MEI_4.0/CRIM_Model_0008.mei']
#corpus = CorpusBase(all_crim)
corpus = CorpusBase(corpus_files)

Requesting file from https://crimproject.org/mei/MEI_4.0/CRIM_Model_0008.mei...
Successfully imported.


# Correct the MEI Metadata

Just as an experiment, we load the MEI files directly in order to adjust metadata. 

This routine fixes the title information.

These changes would be implemented in https://github.com/HCDigitalScholarship/intervals/blob/master/main_objs.py

In [95]:
import xml.etree.ElementTree as ET
import requests

MEINSURI = 'http://www.music-encoding.org/ns/mei'
MEINS = '{%s}' % MEINSURI

for i, path in enumerate(corpus_files):
  if path[0] == '/':
    mei_doc = ET.parse(path)
  else:
    mei_doc = ET.fromstring(requests.get(path).text)
  # Find the title from the MEI file and update the Music21 Score metadata
  title = mei_doc.find(f'{MEINS}meiHead//{MEINS}titleStmt/{MEINS}title').text
  corpus.scores[i].metadata.title = title

In [96]:
for s in corpus.scores:
    print(s.metadata.title)

Ave Maria


***

### Select Actual or Incremental Durations

#### About Rhythmic Durations

- For `find_close_matches` and `find_exact_matches`, rhythmic variation/duration is displayed, but **not** factored into the calculation of matching.
- **Incremental Offset** calculates the intervals using a **fixed offset between notes**, no matter their actual duration.  Use this to ignore passing tones or other ornaments.  The offsets are expressed in multiples of the quarter note (Offset = 1 samples at quarter note; Offset = 2 at half note, etc). Set with `vectors = IntervalBase(corpus.note_list_incremental_offset(2))`

In [90]:
#vectors = IntervalBase(corpus.note_list)
vectors = IntervalBase(corpus.note_list_incremental_offset(2))


***

### Select Generic or Semitone:

- **Length of the Soggetto**: `into_patterns([vectors.semitone_intervals], 5)` 

- The **number** in this command represents the **minimum number of vectors to find**. 5 vectors is 6 notes.


In [91]:
patterns = into_patterns([vectors.generic_intervals], 5)
#patterns = into_patterns([vectors.semitone_intervals], 4)

***

### Select Exact Matches Here
#### (Use comment feature to select screen preview or CSV output) 

- **Exact** is exact in *all* ways `find_exact_matches(patterns, 2)` 
- The **number** in this command represents the **minimum number of matching melodies needed before reporting**. This allows us to filter for common or uncommon soggetti.

In [92]:
exact_matches = find_exact_matches(patterns, 3)
# Use this for exact screen preview
#for item in exact_matches:
    #item.print_exact_matches()

output = export_pandas(exact_matches)
pd.DataFrame(output).head()
#export_to_csv(exact_matches) 

Finding exact matches...
99 melodic intervals had more than 3 exact matches.



,pattern_generating_match,pattern_matched,piece_title,part,start_measure,end_measure,note_durations,ema,ema_url
0,"[1, 4, 1, 1, 1]","[1, 4, 1, 1, 1]",Ave Maria,[Superius],1,2,"[2, 2, 2, 2, 2, 2]","1-2/1/@1.0-end,@start-1.0",https://ema.crimproject.org/https%3A%2F%2Fcrim...
1,"[1, 4, 1, 1, 1]","[1, 4, 1, 1, 1]",Ave Maria,[Superius],101,102,"[2, 2, 2, 2, 2, 2]","101-102/1/@3.0-end,@start-1.0",https://ema.crimproject.org/https%3A%2F%2Fcrim...
2,"[1, 4, 1, 1, 1]","[1, 4, 1, 1, 1]",Ave Maria,[Superius],105,106,"[2, 2, 2, 2, 2, 2]","105-106/1/@3.0-end,@start-1.0",https://ema.crimproject.org/https%3A%2F%2Fcrim...
3,"[1, 4, 1, 1, 1]","[1, 4, 1, 1, 1]",Ave Maria,Altus,3,4,"[2, 2, 2, 2, 2, 2]","3-4/2/@1.0-end,@start-1.0",https://ema.crimproject.org/https%3A%2F%2Fcrim...
4,"[1, 4, 1, 1, 1]","[1, 4, 1, 1, 1]",Ave Maria,Tenor,5,6,"[2, 2, 2, 2, 2, 2]","5-6/3/@1.0-end,@start-1.0",https://ema.crimproject.org/https%3A%2F%2Fcrim...


***

### Select Close Matches Here
#### (Comment out the 'for item iteration' in order to skip screen preview)

- **Close** matches allow for melodic variation (see more below). `find_close_matches(patterns, 2, 1)`
- The **first number** in this command is the **minimum number of melodies** needed before reporting
- The **second number** is **threshold of similarity** needed in order to find a match. 
- Lower number = very similar; higher number = less similar

##### More about Close Matches  
- The **threshold for close matches** is determined by the **second number** called in the method. 
- We select two patterns, then compare *each vector in each pattern successively*. 
- The *differences between each vector are summed*. 
- If that value is **below the threshold specified**, we consider the **two patterns closely matched**.
- The format of the method call is  `find_close_matches(the array you get from into_patterns, minimum matches needed to be displayed, threshold for close match)`.

In [13]:
close_matches = find_close_matches(patterns, 2, 1)
#for item in close_matches:
   #item.print_close_matches()
    #return pd.DataFrame(close_matches)

output = export_pandas(close_matches)
pd.DataFrame(output).head()
#export_to_csv(close_matches)

Finding close matches...
142 melodic intervals had more than 2 exact or close matches.



,pattern_generating_match,pattern_matched,piece_title,part,start_measure,end_measure,note_durations,ema,ema_url
0,"[4, 1, 2, 2, -3]","[4, 1, 2, 2, -3]",Ave Maria,[Superius],1,4,"[4.0, 8.0, 4.0, 4.0, 4.0, 8.0]","1-4/1/@1.0-end,@start-end,@start-end,@start-1.0",https://ema.crimproject.org/https%3A%2F%2Fcrim...
1,"[4, 1, 2, 2, -3]","[4, 1, 2, 2, -3]",Ave Maria,[Superius],105,107,"[4.0, 8.0, 4.0, 4.0, 6.0, 2.0]","105-107/1/@3.0-end,@start-end,@start-3.5",https://ema.crimproject.org/https%3A%2F%2Fcrim...
2,"[4, 1, 2, 2, -3]","[4, 1, 2, 2, -3]",Ave Maria,Altus,3,6,"[4.0, 8.0, 4.0, 4.0, 4.0, 8.0]","3-6/2/@1.0-end,@start-end,@start-end,@start-1.0",https://ema.crimproject.org/https%3A%2F%2Fcrim...
3,"[4, 1, 2, 2, -3]","[4, 1, 2, 2, -3]",Ave Maria,Tenor,5,8,"[4.0, 8.0, 4.0, 4.0, 4.0, 8.0]","5-8/3/@1.0-end,@start-end,@start-end,@start-1.0",https://ema.crimproject.org/https%3A%2F%2Fcrim...
4,"[4, 1, 2, 2, -3]","[4, 1, 2, 2, -3]",Ave Maria,Bassus,7,10,"[4.0, 8.0, 4.0, 4.0, 4.0, 8.0]","7-10/4/@1.0-end,@start-end,@start-end,@start-1.0",https://ema.crimproject.org/https%3A%2F%2Fcrim...


***

### Classify Patterns Here 
#### Note:  depends on choice of Close or Exact above!  Must choose appropriate one below!
#### No Pandas Preview, but CSV Export OK
#### Scroll to bottom of output to name and save CSV file

In [24]:
classify_matches(close_matches, 2)
#classify_matches(exact_matches, 2)
classified_matches = classify_matches(close_matches, 2)
#pd.DataFrame(classified_matches)
#output = export_pandas(classified_matches)
#pd.DataFrame(output).head()

## For CSV export, use the following (and follow prompts for file name)

#export_to_csv(classified_matches)

periodic_entry:
Pattern: [4, 1, 2, 2, -3], Locations in entry: 
- Measure 1 in voice 1
- Measure 3 in voice 2
- Measure 5 in voice 3
- Measure 7 in voice 4
fuga:
Pattern: [4, -2, -2, 2, 2], Locations in entry: 
- Measure 10 in voice 1
- Measure 12 in voice 2
- Measure 14 in voice 3
imitative duo:
Pattern: [2, 2, -3, -2, -2], Locations in entry: 
- Measure 55 in voice 1
- Measure 56 in voice 2
- Measure 60 in voice 3
- Measure 61 in voice 4
fuga:
Pattern: [2, -2, -2, -2, -2], Locations in entry: 
- Measure 41 in voice 1
- Measure 41 in voice 3
- Measure 94 in voice 2
- Measure 98 in voice 2
fuga:
Pattern: [1, 2, 2, 1, 2], Locations in entry: 
- Measure 31 in voice 1
- Measure 35 in voice 3
- Measure 94 in voice 1
- Measure 94 in voice 3
imitative duo:
Pattern: [1, 1, 2, 1, 2], Locations in entry: 
- Measure 94 in voice 1
- Measure 94 in voice 3
- Measure 98 in voice 1
- Measure 98 in voice 3
imitative duo:
Pattern: [2, 2, 1, 2, -2], Locations in entry: 
- Measure 32 in voice 1
- Measure

periodic_entry:
Pattern: [4, 1, 2, 2, -3], Locations in entry: 
- Measure 1 in voice 1
- Measure 3 in voice 2
- Measure 5 in voice 3
- Measure 7 in voice 4
fuga:
Pattern: [4, -2, -2, 2, 2], Locations in entry: 
- Measure 10 in voice 1
- Measure 12 in voice 2
- Measure 14 in voice 3
imitative duo:
Pattern: [2, 2, -3, -2, -2], Locations in entry: 
- Measure 55 in voice 1
- Measure 56 in voice 2
- Measure 60 in voice 3
- Measure 61 in voice 4
fuga:
Pattern: [2, -2, -2, -2, -2], Locations in entry: 
- Measure 41 in voice 1
- Measure 41 in voice 3
- Measure 94 in voice 2
- Measure 98 in voice 2
fuga:
Pattern: [1, 2, 2, 1, 2], Locations in entry: 
- Measure 31 in voice 1
- Measure 35 in voice 3
- Measure 94 in voice 1
- Measure 94 in voice 3
imitative duo:
Pattern: [1, 1, 2, 1, 2], Locations in entry: 
- Measure 94 in voice 1
- Measure 94 in voice 3
- Measure 98 in voice 1
- Measure 98 in voice 3
imitative duo:
Pattern: [2, 2, 1, 2, -2], Locations in entry: 
- Measure 32 in voice 1
- Measure

## Read CSV of Classified Matches

- Update file name to match the output of previous cells for Classifier

In [ ]:
pd.read_csv('Sandrin_Soggetti.csv', usecols=['Pattern Generating Match', 'Classification Type', 'Soggetti 1 Part', 'Soggetti 1 Measure', 'Soggetti 2 Part', 'Soggetti 2 Measure', 'Soggetti 3 Part', 'Soggetti 3 Measure', 'Soggetti 4 Part', 'Soggetti 4 Measure'])


In [68]:
pd.DataFrame(classified_matches)

In [69]:
out

,0
0,<crim_intervals.main.ClassifiedMatch object at...
1,<crim_intervals.main.ClassifiedMatch object at...
2,<crim_intervals.main.ClassifiedMatch object at...
3,<crim_intervals.main.ClassifiedMatch object at...
4,<crim_intervals.main.ClassifiedMatch object at...
...,...
132,<crim_intervals.main.ClassifiedMatch object at...
133,<crim_intervals.main.ClassifiedMatch object at...
134,<crim_intervals.main.ClassifiedMatch object at...
135,<crim_intervals.main.ClassifiedMatch object at...


In [67]:
df.head()

,column1,column2
row1,NaN,NaN
row2,NaN,NaN


In [85]:
def classified_pandas(matches):
    import pandas as pd
    classified_data = []
    for classified_matches in matches:
            classified_dict = {
              #"Piece": match.first_note.metadata.title,
              "Pattern_Generating_Match": classified_matches.pattern,
              "Classification_Type": classified_matches.type,
              "Soggetto": soggetti.first_note.part
               
              }        
        
            classified_data.append(classified_dict)
    return pd.DataFrame(classified_data)

In [86]:
classified_pandas(classified_matches)

NameError: name 'soggetti' is not defined

In [45]:
classified_data

NameError: name 'classified_data' is not defined